In [4]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

#function definitions
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def make_line(x, y, xmax, ymax, slope):
    c = y-slope*(x)
    ystart=ymax;
    #y = slope*x+c
    xstart = (ymax-c)/slope
    yend = 325
    xend = (yend-c)/slope
    return [xstart, ystart, xend, yend]
    
def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros(img.shape, dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def process_image(image):
    # Grab the x and y size and make a copy of the image
    ysize = image.shape[0]
    xsize = image.shape[1]
    x_l=0;
    x_r=0;
    y_l=0;
    y_r=0;
    xmax_l=0;
    ymax_l=0;
    xmax_r=0;
    ymax_r=0;
    xmin_l=1000;
    ymin_l=1000;
    xmin_r=1000;
    ymin_r=1000;
    lslope_t=[]
    rslope_t=[]

    #now we colorize the surprize 
    gray = grayscale(image)

    # Define a kernel size and apply Gaussian smoothing
    kernel_size = 9
    blur_gray = gaussian_blur(gray, kernel_size)

    # Define our parameters for Canny and apply
    low_threshold = 50
    high_threshold = 150
    edges = canny(blur_gray, low_threshold, high_threshold)

    # Next we'll create a masked edges image using cv2.fillPoly()
    mask = np.zeros_like(edges)   
    ignore_mask_color = 255   

    # This time we are defining a four sided polygon to mask
    vertices = np.array([[(0,ysize),(xsize/4+xsize/8, ysize/2+ysize/8), (3*xsize/4-xsize/8, ysize/2+ysize/8), (xsize,ysize)]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color) #fillPoly will fill the area bounded by the polygons prescribed
    masked_edges = cv2.bitwise_and(edges, mask)


    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 15     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 40 #minimum number of pixels making up a line
    max_line_gap = 20    # maximum gap in pixels between connectable line segments
    line_image = np.copy(image)*0 # creating a blank to draw lines on
    line_image2 = np.copy(image)*0 # creating a blank to draw lines on

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),
                                min_line_length, max_line_gap)

    i=0
    # Iterate over the output "lines" and draw lines on a blank image
    for line in lines:
        for x1,y1,x2,y2 in line:
            #calculate the slope and get the lines from hough transform
            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)
            if(x1<xsize/2):
                lslope_t.append((y2-y1)/(x2-x1))
                x_l=x1
                y_l=y1
            if(x1>xsize/2):
                rslope_t.append((y2-y1)/(x2-x1))
                x_r=x1
                y_r=y1
            i+=1

    #this converts the slope list into an array        
    l_slope=np.array(lslope_t)
    r_slope=np.array(rslope_t)

    #this will find the mean of the array to get average slope
    lslope=np.mean(l_slope)
    rslope=np.mean(r_slope)

    [xstart_l, ystart_l, xend_l, yend_l] = make_line(x_l, y_l, xsize, ysize, lslope)
    [xend_r, yend_r, xstart_r, ystart_r] = make_line(x_r, y_r, xsize, ysize, rslope)

    if not (math.isnan(xstart_l) or math.isnan(xstart_r) or math.isnan(ystart_l) or math.isnan(ystart_r) or math.isinf(xstart_l) or math.isinf(xstart_r) or math.isinf(ystart_l) or math.isinf(ystart_r)):
        cv2.line(line_image2,(int(xstart_l), int(ystart_l)),(int(xend_l), int(yend_l)),(255,0,0),10)
        cv2.line(line_image2,(int(xstart_r), int(ystart_r)),(int(xend_r), int(yend_r)),(255,0,0),10)


    # Create a "color" binary image to combine with line image
    color_edges = np.dstack((edges, edges, edges)) 

    # Draw the lines on the edge image
    lines_edges = cv2.addWeighted(image, 1, line_image, 0.8, 0) 
    lines_crop = region_of_interest(lines_edges,vertices)

    # Draw the lines on the edge image
    lines_edges2 = cv2.addWeighted(image, 1, line_image2, 0.8, 0) 
    return lines_edges2

In [ ]:
#start of program
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)
plt.show()

image_out= process_image(image)
plt.imshow(image_out) 
plt.show()

print(image_out.shape)
print(image.shape)

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

In [60]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))